In [1]:
from importlib.util import find_spec
%load_ext autoreload
%autoreload 2

%matplotlib inline

if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('../..')

In [2]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.models import QCNNCong
from tensorflow.keras.callbacks import ReduceLROnPlateau
import argparse
import numpy as np

import matplotlib.pyplot as plt
import time

In [20]:
args = argparse.Namespace()

# Data
args.pca = 9
args.normalize = 1
args.binary_data = [0,1]
args.batch_size = 256
args.validation_split = 0.05
args.hinge_labels = 1
args.use_quantum = 1

# Base Model
args.learning_rate = 1e-3
args.epochs = 5

# Quantum CNN Parameters
args.n_layers = 3

In [21]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Performing PCA on data...
Cumulative sum on train : 68.27599655429124
Cumulative sum on test: 68.27599655429124
Normalizing data...
Hinge labels...
Hinge labels...

Dataset :MNIST
╒════════╤══════════════════╤════════════════╤═════════════════╤═══════════╕
│ Data   │ Train size       │ Val size       │ Test size       │ Dims      │
╞════════╪══════════════════╪════════════════╪═════════════════╪═══════════╡
│ X      │ (12031, 3, 3, 1) │ (634, 3, 3, 1) │ (2115, 3, 3, 1) │ (3, 3, 1) │
├────────┼──────────────────┼────────────────┼─────────────────┼───────────┤
│ y      │ (12031,)         │ (634,)         │ (2115,)         │ (1,)      │
╘════════╧══════════════════╧════════════════╧═════════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═══════╪══════════════════════════╡
│ Train Ima

In [22]:
data.config()

{'input_dims': (3, 3, 1), 'output_dims': (1,), 'mapping': [0, 1]}

In [23]:
model = QCNNCong(data.config(), args)

../../qml_hep_lhc/layers/qconv2d.py:61: UserWarning: n_qubits is only used for NQubit ansatz.
  warnings.warn('n_qubits is only used for NQubit ansatz.')


In [24]:
model.compile()
model.build_graph().summary()

Model: "QCNNCong"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 3, 3, 1)]         0         
                                                                 
 qconv2d (QConv2D)           (None, 1, 1, 1, 1)        81        
                                                                 
 flatten_14 (Flatten)        (None, 1)                 0         
                                                                 
Total params: 81
Trainable params: 72
Non-trainable params: 9
_________________________________________________________________


In [25]:
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                              factor=np.sqrt(0.1),
                                              patience=5,
                                              min_delta=0.0001,
                                              min_lr=1e-8)
callbacks = [lr_scheduler_callback]

In [26]:
model.fit(data, callbacks)

Epoch 1/5
47/47 [==============================] - 36s 772ms/step - loss: 0.9051 - q_auc_2: 0.8394 - custom_accuracy: 0.6566 - val_loss: 0.8034 - val_q_auc_2: 0.9908 - val_custom_accuracy: 0.8138 - lr: 0.0010
Epoch 2/5
47/47 [==============================] - 36s 776ms/step - loss: 0.7045 - q_auc_2: 0.9946 - custom_accuracy: 0.8998 - val_loss: 0.6012 - val_q_auc_2: 0.9996 - val_custom_accuracy: 0.9775 - lr: 0.0010
Epoch 3/5
47/47 [==============================] - 38s 805ms/step - loss: 0.5408 - q_auc_2: 0.9991 - custom_accuracy: 0.9896 - val_loss: 0.4872 - val_q_auc_2: 1.0000 - val_custom_accuracy: 0.9987 - lr: 0.0010
Epoch 4/5
47/47 [==============================] - 38s 821ms/step - loss: 0.4597 - q_auc_2: 0.9993 - custom_accuracy: 0.9945 - val_loss: 0.4202 - val_q_auc_2: 1.0000 - val_custom_accuracy: 0.9987 - lr: 0.0010
Epoch 5/5
47/47 [==============================] - 37s 778ms/step - loss: 0.3886 - q_auc_2: 0.9993 - custom_accuracy: 0.9945 - val_loss: 0.3397 - val_q_auc_2: 1.000

In [27]:
model.test(data, callbacks=callbacks)

9/9 [==============================] - 4s 490ms/step - loss: 0.3415 - q_auc_2: 0.9998 - custom_accuracy: 0.9965


[0.3415096700191498,
 0.9998485445976257,
 0.9965277910232544,
 0.9998485445976257]

In [31]:
args = argparse.Namespace()

# Data
args.use_quantum = 1

# Base Model
args.learning_rate = 1e-3
args.epochs = 5

# Quantum CNN Parameters
args.n_layers = 4
args.cluster_state = 1

In [32]:
model = QCNNCong(data.config(), args)
model.compile()
model.build_graph().summary()

Model: "QCNNCong"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 3, 3, 1)]         0         
                                                                 
 qconv2d (QConv2D)           (None, 1, 1, 1, 1)        81        
                                                                 
 flatten_276 (Flatten)       (None, 1)                 0         
                                                                 
Total params: 81
Trainable params: 72
Non-trainable params: 9
_________________________________________________________________


../../qml_hep_lhc/layers/qconv2d.py:61: UserWarning: n_qubits is only used for NQubit ansatz.
  warnings.warn('n_qubits is only used for NQubit ansatz.')


In [33]:
model.fit(data, callbacks)

Epoch 1/5
47/47 [==============================] - 38s 801ms/step - loss: 0.6791 - q_auc_3: 0.9866 - custom_accuracy: 0.9282 - val_loss: 0.5324 - val_q_auc_3: 0.9983 - val_custom_accuracy: 0.9761 - lr: 0.0010
Epoch 2/5
47/47 [==============================] - 37s 787ms/step - loss: 0.4686 - q_auc_3: 0.9995 - custom_accuracy: 0.9861 - val_loss: 0.3839 - val_q_auc_3: 0.9999 - val_custom_accuracy: 0.9961 - lr: 0.0010
Epoch 3/5
47/47 [==============================] - 36s 775ms/step - loss: 0.3071 - q_auc_3: 0.9995 - custom_accuracy: 0.9925 - val_loss: 0.2265 - val_q_auc_3: 1.0000 - val_custom_accuracy: 0.9987 - lr: 0.0010
Epoch 4/5
47/47 [==============================] - 37s 789ms/step - loss: 0.1990 - q_auc_3: 0.9995 - custom_accuracy: 0.9940 - val_loss: 0.1600 - val_q_auc_3: 1.0000 - val_custom_accuracy: 1.0000 - lr: 0.0010
Epoch 5/5
47/47 [==============================] - 37s 796ms/step - loss: 0.1558 - q_auc_3: 0.9996 - custom_accuracy: 0.9945 - val_loss: 0.1337 - val_q_auc_3: 1.000

In [34]:
model.test(data, callbacks=callbacks)

9/9 [==============================] - 4s 489ms/step - loss: 0.1383 - q_auc_3: 1.0000 - custom_accuracy: 0.9944


[0.13828720152378082,
 0.9999609589576721,
 0.9944354295730591,
 0.9999609589576721]